In [1]:
import tensorflow.keras as keras
import numpy as np
import pandas as pd

In [2]:
from keras.layers import Input, ZeroPadding2D, Conv2D, BatchNormalization, Activation, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, Flatten, Dense, Add
from keras.models import Model
from keras.initializers import glorot_uniform
from keras.utils import plot_model
from keras.utils.vis_utils import model_to_dot
from IPython.display import SVG

In [3]:
def convert_one_hot_encod(Y):
    df = pd.DataFrame(Y)
    df = df.astype(str, errors='raise')
    df = pd.get_dummies(df)
    Y = df.to_numpy()
    
    return Y
    

In [4]:
# X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()

X_train = X_train/255.
X_test = X_test/255.

y_train = convert_one_hot_encod(y_train)
y_test = convert_one_hot_encod(y_test)

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(y_test.shape))

170500096/170498071 [==============================] - 3s 0us/step
number of training examples = 50000
number of test examples = 10000
X_train shape: (50000, 32, 32, 3)
Y_train shape: (50000, 10)
X_test shape: (10000, 32, 32, 3)
Y_test shape: (10000, 10)


In [5]:
def bottleneck_with_identity_shortcut(X, filters, f, stage, block):
    
        conv_name_base = 'res' + str(stage) + block + '_branch'
        bn_name_base = 'bn' + str(stage) + block + '_branch'
    
        X_shortcut = X
    
        X = Conv2D(filters[0], kernel_size=(1,1), strides=(1,1), padding="valid", name = conv_name_base + '2a' ,kernel_initializer = glorot_uniform(seed=0))(X)
        X = BatchNormalization(axis=3)(X)
        X = Activation("relu")(X)
        
        X = Conv2D(filters[1], kernel_size=(f,f), strides=(1,1), padding="same", name = conv_name_base + '2b' ,kernel_initializer = glorot_uniform(seed=0))(X)
        X = BatchNormalization(axis=3)(X)
        X = Activation("relu")(X)
        
        X = Conv2D(filters[2], kernel_size=(1,1), strides=(1,1), padding="valid", name = conv_name_base + '2c' ,kernel_initializer = glorot_uniform(seed=0))(X)
        X = BatchNormalization(axis=3)(X)
        
        X = Add()([X, X_shortcut])
        X = Activation("relu")(X)
        
        return X

In [6]:
def bottleneck_with_projection_shortcut(X, filters, f, s, stage, block):
    
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    X_shortcut = X
    
    X = Conv2D(filters[0], kernel_size=(1,1), strides=(s,s), padding="valid", name = conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation("relu")(X)
    
    X = Conv2D(filters[1], kernel_size=(f,f), strides=(1,1), padding="same", name = conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation("relu")(X)
    
    X = Conv2D(filters[2], kernel_size=(1,1), strides=(1,1), padding="valid", name = conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    
    X_shortcut = Conv2D(filters[2], kernel_size=(1,1), strides=(s,s), padding = 'valid', name = conv_name_base + '1', kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut)
    
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X) 
    
    return X

In [7]:
def res_net_50(input_shape, classes):
    
    X_input = Input(input_shape)
    
    X = ZeroPadding2D((4, 4))(X_input)
    
    X = Conv2D(64, kernel_size=(3,3), strides=(2,2),padding="valid", name="conv1", kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name="bn_conv1")(X)
    X = Activation("relu")(X)
    X = MaxPooling2D((2,2), strides=(2,2))(X)
    
    X = bottleneck_with_identity_shortcut(X, [32,64,64], f=3, stage=2, block='a')
    X = bottleneck_with_identity_shortcut(X, [32,64,64], f=3, stage=2, block='b')
    X = bottleneck_with_identity_shortcut(X, [32,64,64], f=3, stage=2, block='c')
    
    X = bottleneck_with_projection_shortcut(X, [32,32,128], f=3, s=2, stage=3, block='a')
    X = bottleneck_with_identity_shortcut(X, [64,64,128], f=3, stage=3, block='b')
    X = bottleneck_with_identity_shortcut(X, [64,64,128], f=3, stage=3, block='c')
    X = bottleneck_with_identity_shortcut(X, [64,64,128], f=3, stage=3, block='d')
    
    X = bottleneck_with_projection_shortcut(X, [64,64,256], f=3, s=2, stage=4, block='a')
    X = bottleneck_with_identity_shortcut(X, [128,128,256], f=3, stage=4, block='b')
    X = bottleneck_with_identity_shortcut(X, [128,128,256], f=3, stage=4, block='c')
    X = bottleneck_with_identity_shortcut(X, [128,128,256], f=3, stage=4, block='d')
    X = bottleneck_with_identity_shortcut(X, [128,128,256], f=3, stage=4, block='e')
    X = bottleneck_with_identity_shortcut(X, [128,128,256], f=3, stage=4, block='f')
    
    X = bottleneck_with_projection_shortcut(X, [128,128,512], f=3, s=2, stage=5, block='a')
    X = bottleneck_with_identity_shortcut(X, [256,256,512], f=3, stage=5, block='b')
    X = bottleneck_with_identity_shortcut(X, [256,256,512], f=3, stage=5, block='c')
                                                                                                                  
#     X = AveragePooling2D((2, 2), strides=(2,2), name='avg_pool')(X)
    X =GlobalAveragePooling2D()(X)
                                                                                                                  
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
                                                                                                                  
    model = Model(inputs = X_input, outputs = X, name='ResNet50')
                                                                                                                  
    return model
    

In [13]:
model = res_net_50(input_shape = (32, 32, 3), classes = 10)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
model.fit(X_train, y_train, epochs = 40, batch_size = 128)

Epoch 1/40
391/391 [==============================] - 15s 27ms/step - loss: 2.0013 - accuracy: 0.3101
Epoch 2/40
391/391 [==============================] - 10s 26ms/step - loss: 1.3744 - accuracy: 0.5045
Epoch 3/40
391/391 [==============================] - 10s 26ms/step - loss: 1.2478 - accuracy: 0.5619
Epoch 4/40
391/391 [==============================] - 11s 27ms/step - loss: 1.0670 - accuracy: 0.6222
Epoch 5/40
391/391 [==============================] - 10s 26ms/step - loss: 0.9269 - accuracy: 0.6724
Epoch 6/40
391/391 [==============================] - 10s 26ms/step - loss: 0.8394 - accuracy: 0.7038
Epoch 7/40
391/391 [==============================] - 11s 28ms/step - loss: 0.7842 - accuracy: 0.7254
Epoch 8/40
391/391 [==============================] - 10s 27ms/step - loss: 0.6840 - accuracy: 0.7620
Epoch 9/40
391/391 [==============================] - 10s 27ms/step - loss: 0.5878 - accuracy: 0.7938
Epoch 10/40
391/391 [==============================] - 11s 27ms/step - loss: 0.563

In [15]:
preds = model.evaluate(X_test, y_test)

313/313 [==============================] - 3s 8ms/step - loss: 1.5353 - accuracy: 0.7124


In [16]:
print("Loss : {0:.5f}".format(preds[0]))
print("Accuracy : {0:.5f}".format(preds[1]))

Loss : 1.53525
Accuracy : 0.71240


In [17]:
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 40, 40, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 19, 19, 64)   1792        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 19, 19, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

In [ ]:
plot_model(model, to_file='model.png')
SVG(model_to_dot(model).create(prog='dot', format='svg'))